<a href="https://colab.research.google.com/github/natalie-ayers/Classification-of-Twitter-Vaccine-Hesitancy/blob/main/LSC_assignment_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
! pip install mpi4py

     |████████████████████████████████| 2.4 MB 5.3 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for mpi4py: filename=mpi4py-3.1.1-cp37-cp37m-linux_x86_64.whl size=2180572 sha256=75314f8f272043d95a2097b912051fd1adaae2dc441c2b87f0d78fd0bdbd0307
  Stored in directory: /root/.cache/pip/wheels/91/be/c0/2b0347be1de5cd8ca9fe67da7ec8c3fe8930fcb6b0df6f2255
Successfully built mpi4py


In [19]:
%%writefile health_sim_orig.py 

import numpy as np
import scipy.stats as sts
import time


t0 = time.time()

# Set model parameters
rho = 0.5
mu = 3.0
sigma = 1.0
z_0 = mu

# Set simulation parameters, draw all idiosyncratic random shocks,
# and create empty containers
S = 1000 # Set the number of lives to simulate
T = int(4160) # Set the number of periods for each simulation
np.random.seed(25)
eps_mat = sts.norm.rvs(loc=0, scale=sigma, size=(T, S))
z_mat = np.zeros((T, S))

for s_ind in range(S):
  z_tm1 = z_0
  for t_ind in range(T):
    e_t = eps_mat[t_ind, s_ind]
    z_t = rho * z_tm1 + (1 - rho) * mu + e_t
    z_mat[t_ind, s_ind] = z_t
    z_tm1 = z_t

time_elapsed = time.time() - t0

print("Simulated %d Lifecycles: %f seconds" \
                % (S, time_elapsed))

Overwriting health_sim_orig.py


In [20]:
! mpirun --allow-run-as-root -n 4 python health_sim_orig.py

Simulated 1000 Lifecycles: 18.934609 seconds
Simulated 1000 Lifecycles: 18.800377 seconds
Simulated 1000 Lifecycles: 19.016541 seconds
Simulated 1000 Lifecycles: 19.212811 seconds


In [ ]:
%%writefile mpi_rand_walk.py 
from mpi4py import MPI
import numpy as np
import scipy.stats as sts
import time

def sim_rand_walks_parallel(n_runs):

    for s_ind in range(S):
      z_tm1 = z_0
      for t_ind in range(T):
        e_t = eps_mat[t_ind, s_ind]
        z_t = rho * z_tm1 + (1 - rho) * mu + e_t
        z_mat[t_ind, s_ind] = z_t
        z_tm1 = z_t

    return

def main():
    # Set model parameters
    rho = 0.5
    mu = 3.0
    sigma = 1.0
    z_0 = mu
    
    # Set simulation parameters, draw all idiosyncratic random shocks,
    # and create empty containers
    S = 1000 # Set the number of lives to simulate
    T = int(4160) # Set the number of periods for each simulation
    np.random.seed(25)
    eps_mat = sts.norm.rvs(loc=0, scale=sigma, size=(T, S))
    z_mat = np.zeros((T, S))

    sim_rand_walks_parallel(n_runs=10000)

if __name__ == '__main__':
    main()

In [7]:
import numpy as np
import scipy.stats as sts

sigma = 1.0
S = 1000 # Set the number of lives to simulate
T = int(4160) # Set the number of periods for each simulation
np.random.seed(25)

eps_mat = sts.norm.rvs(loc=0, scale=sigma, size=(T, S))
eps_mat[1,2]

-0.7348711020538244

In [11]:
z_mat = np.zeros((T, S))
z_mat.shape

(4160, 1000)